In [31]:
import os
import sys
import numpy as np
import skimage
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
import mrcnn.model as modellib
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [32]:
# minimum input size = 128
class ShapesConfig(Config):
    # Give the configuration a recognizable name
    NAME = "skin_stephen"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 16
    NUM_CLASSES = 1 + 2  # background + 2 types
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128
    RPN_ANCHOR_SCALES = (16,32,64,128,256)  # anchor side in pixels
    TRAIN_ROIS_PER_IMAGE = 8
    STEPS_PER_EPOCH = 626 // IMAGES_PER_GPU
    VALIDATION_STEPS = 626 // IMAGES_PER_GPU
    LEARNING_RATE = 0.001
    USE_MINI_MASK = False
config = ShapesConfig()

class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MAX_DIM = 128
inference_config = InferenceConfig()

In [33]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference",
                          config=inference_config,
                          model_dir=MODEL_DIR)

model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

keras version 2.2.5
tf version 1.12.0
Loading weights from  C:\Users\kuki\PycharmProjects\Mask_RCNN\logs\skin_stephen20201101T0139\mask_rcnn_skin_stephen_0025.h5
Re-starting from epoch 25


In [34]:
from PIL import Image
from skimage import io

## put folder path here to apply your model to classify
src = r'\\motherserverdw\Kyu_Sync\server for undergrads\Stephen\train\Wirtz.Denis_OTS-19_5021-006_tile_0\image'
##
dst = os.path.join(src,'classified')
if not os.path.exists(dst): os.mkdir(dst)

images = [os.path.join(src,_) for _ in os.listdir(src) if _.endswith('png')]
idd = []
for original_image in images:
    original_image2 = skimage.io.imread(original_image)
    results = model.detect([original_image2], verbose=1)
    r = results[0]
    masks = r['masks']
    masks = np.moveaxis(masks,2,0)
    if len(masks)<1:
        continue
    maskzero=np.zeros(masks[0].shape)
    for mask,id in zip(masks,r['class_ids']):
        idd.append(id)
        maskzero[mask]=id
    im = Image.fromarray(maskzero)
    im.save(os.path.join(dst, os.path.basename(original_image).replace('png','tif')))

Processing 1 images
image                    shape: (100, 100, 3)         min:   55.00000  max:  255.00000  uint8
molded_images            shape: (1, 128, 128, 3)      min:  -60.80000  max:  151.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  128.00000  float64
anchors                  shape: (1, 4092, 4)          min:   -1.42535  max:    1.92141  float32
Processing 1 images
image                    shape: (100, 100, 3)         min:   27.00000  max:  255.00000  uint8
molded_images            shape: (1, 128, 128, 3)      min:  -81.80000  max:  150.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  128.00000  float64
anchors                  shape: (1, 4092, 4)          min:   -1.42535  max:    1.92141  float32
Processing 1 images
image                    shape: (100, 100, 3)         min:   30.00000  max:  255.00000  uint8
molded_images            shape: (1, 128, 128, 3)      min:  -86.80000  max:  150.1

In [35]:
# calculate accuracy of the model based on predicted label
src = r'\\motherserverdw\Kyu_Sync\server for undergrads\Stephen\train\Wirtz.Denis_OTS-19_5021-006_tile_0\image\classified'
images = [os.path.join(src,_) for _ in os.listdir(src) if _.endswith('tif')]
if 'tile' in src:
    labels = [1]*len(images)
else: labels = [2]*len(images)
predictedlabels = [np.unique(Image.open(_))[-1] for _ in images]
a = np.equal(labels,predictedlabels)
acc = np.sum(a)/len(a)*100
acc

3.787878787878788